In [22]:
from pyspark.mllib.clustering import KMeans
from numpy import array
from math import sqrt
from pyspark.sql import SQLContext
import json
#import ConfigParser
import sys
#import random
import os
#import numpy as np
import calendar
import time
#import urllib
#import urllib2
import base64
import requests
import math

In [23]:
#Globals for Kafka config
OUTPUTTOPIC='newOutput'
URL = 'http://134.168.20.191:9443/topics/' + OUTPUTTOPIC
HEADERS = {'Content-Type': 'application/vnd.kafka.v1+json'}

# x mins ago of the current time
min_offset = 240

#Epoch time 5 mins ago as of the current time
newEpochOffset = (calendar.timegm(time.gmtime()) - min_offset * 60) * 1000

#last N entries in the database
limitN = 20

#Sensors to analyze
sensors = ['PM41302', 'PM41802', 'PM107631']

In [24]:
def do_the_math(SensorID):
    #df1 = sqlContext.sql("SELECT codigo, intensidad, velocidad, tf, dt FROM madridtraffic WHERE ts > " + str(newEpochOffset))
    df1 = sqlContext.sql("SELECT codigo, intensidad, velocidad, tf, dt FROM madridtraffic order by ts desc limit " + str(limitN))
    #accessing data of one sensor
    df2 = df1[df1['codigo'] == SensorID]
    print df2.show()
    #mllib clustering function requires data in array and float valuesdf1
    parsedData = df2.map(lambda row: array([float(row.intensidad), float(row.velocidad)]))
    print('parsedData is mapped')
    
    #calculate mean for standardization
    mean = parsedData.mean()
    #print mean
    #calculate standard deviation
    std = parsedData.stdev()

    processedData = parsedData.map(lambda x: ((x-mean)/std))
    print 'normalized the data'

    ## Aren't we blowing up this data each iteration?  Do we really care?
    # Build the model (cluster the data)
    clusters = KMeans.train(processedData, 2, maxIterations=10, runs=1, initializationMode="random")
    print 'clusters initilized'


    ##What is this really doing?
    #denormalize the centers.
    centers = (clusters.centers*std)+mean
    print 'denormalized the centers'


    #midpoints-threshold
    a = (centers[0][0]+centers[1][0])/2
    b = (centers[0][1]+centers[1][1])/2

    #creates dictionary
    message = {"codigo": SensorID, "newThresholds":{"ThresholdSpeed":int(b), "ThresholdIntensity":int(a)}}
    print message
    message = base64.b64encode(str(message))

    #creates dictionary
    messages = {"records":[{"value":message}]}
    #converts dictionary into json msg
    Threshold = json.dumps(messages)

    #fullTopic = URL+topic
    #print fullTopic + " Time: "+ timeofday
    #publish the message: payload is threshold and other field is topic
    r = requests.post(URL, data=Threshold, headers=HEADERS)
    print(r.text)

In [25]:
def kick_off_routine():
    #Loop through the sensors

    for i in sensors:
        do_the_math(i)


In [26]:
df = sqlContext.read.parquet("swift://DataServices.spark/DataServices/TDataServices/dt=2015-11-10")
#print df.show()

df = df.cache()
df.registerTempTable("madridtraffic")

#df_sensors = sqlContext.sql("SELECT codigo FROM madridtraffic WHERE dt > '2015-11-01'")

kick_off_routine()
df.unpersist()

+------+----------+---------+--+--+
|codigo|intensidad|velocidad|tf|dt|
+------+----------+---------+--+--+
+------+----------+---------+--+--+

None
parsedData is mapped
normalized the data


Py4JJavaError: An error occurred while calling o103.trainKMeansModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 38.0 failed 10 times, most recent failure: Lost task 0.9 in stage 38.0 (TID 10849, yp-spark-dal09-env5-0039): java.lang.ArrayIndexOutOfBoundsException: Array index out of range: 0
	at org.apache.spark.mllib.clustering.KMeans$$anonfun$11.apply(KMeans.scala:232)
	at org.apache.spark.mllib.clustering.KMeans$$anonfun$11.apply(KMeans.scala:228)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$17.apply(RDD.scala:686)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$17.apply(RDD.scala:686)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:277)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:244)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:70)
	at org.apache.spark.scheduler.ShuffleMa